# The Issue Description

Let's consider following:

- We have a food delivery service and we're able to provide it for different restaurants in New York.
- We need couriers for it and there are three possible delivery types: couriers of our service, restaurant's couriers (in case if they have their own delivery) and our partners from some taxi service.
- We have our own sales and support managers. They are able to manage some data related to restaurants manually via web-services and it's a part of some common infrastructure including DWH, BI tools, etc. Let's consider that all ETL processes don't fail and we can fully observe our service.
- Right now we're interested only in such entities as restaurants and its orders.

In [392]:
import pandas as pd 
import xxhash as xx 
import h3
import numpy as np

#sample of restaurants which are partners of our delivery service
df_restaurants = pd.read_excel("sample_restaurants.xlsx")
#sample of orders from these restaurants
df_orders = pd.read_excel("sample_orders.xlsx")

In [393]:
df_restaurants.dtypes

id                  int64
name               object
address            object
lat               float64
lon               float64
city               object
delivery_type      object
enabled_status     object
dtype: object

In [394]:
df_restaurants[:3]

,id,name,address,lat,lon,city,delivery_type,enabled_status
0,101504,The Smith,55 3rd Ave,40.73182,-73.98871,New York,restaurant courier,enabled
1,114640,The Smith,55 Third Ave,40.73182,-73.98871,New York,our courier,enabled
2,114872,The Smith's,55 3rd Ave,40.73182,-73.98871,New York,taxi courier,enabled


In [395]:
df_orders.dtypes

id                        int64
order_date       datetime64[ns]
order_amount            float64
restaurant_id             int64
dtype: object

In [396]:
df_orders[:3]

,id,order_date,order_amount,restaurant_id
0,1035539,2022-08-01,68.16,101502
1,1523336,2022-08-01,64.53,101502
2,1807144,2022-08-01,50.04,101502


We found out that one primary key in restaurant's entity means not physically restaurant but the restaurant plus its delivery type: in case if restaurant has its own couriers and we propose our courier service we have to sign two different working agreement with two different legal entities. 

Like in the following example.

In [397]:
#udf only for printing
def print_data(df, i:int)->str:
    print (df.at[i, "id"], df.at[i, "name"], df.at[i, "address"], df.at[i, "city"], df.at[i, "lat"], df.at[i, "lon"], df.at[i, "delivery_type"], df.at[i, "enabled_status"], sep = "; ")

In [398]:
print_data(df_restaurants,9)
print_data(df_restaurants,10)

105964; Bubby's; 120 Hudson St; NY; 40.7227624952702; -74.0084837478272; restaurant courier; enabled
108235; Bubby's; 120 Hudson Street; NY; 40.7227624952702; -74.0084837478272; our courier; enabled


As it was mentioned above, the same restaurant using two different delivery types is allocated to different ids.

Moreover, after some analysis we figure out quite common problem of manually edited systems: the same entities (restaurants in this case) can have a bit different corresponding data. 

Some differences in names:

In [399]:
print_data(df_restaurants,0)
print_data(df_restaurants,2)

101504; The Smith; 55 3rd Ave; New York; 40.73182; -73.98871; restaurant courier; enabled
114872; The Smith's; 55 3rd Ave; New York; 40.73182; -73.98871; taxi courier; enabled


Some differences in addresses:

In [400]:
print_data(df_restaurants,1)
print_data(df_restaurants,2)

114640; The Smith; 55 Third Ave; New York; 40.73182; -73.98871; our courier; enabled
114872; The Smith's; 55 3rd Ave; New York; 40.73182; -73.98871; taxi courier; enabled


Some differences in coordinates:

In [401]:
print_data(df_restaurants,4)
print_data(df_restaurants,3)

102931; The Smith; 956 2nd Ave; New York, NY; 40.75788413292; -73.96870317501; our courier; enabled
121805; The Smith; 956 2nd Ave; New York; 40.7578841329251; -73.9687031750197; restaurant courier; enabled


Basically it's a case of the same restaurants with different delivery types but in case of changing some data (name, address, etc.) the existing row with data would not be updated but other row would be inserted as well as old id would be deactivated. We can notice it according to some differences in current workable statuses:

In [402]:
print_data(df_restaurants,8)
print_data(df_restaurants,16)
print_data(df_restaurants,17)
print_data(df_restaurants,18)

102845; Rubirosa Bar; 235 Mulberry St; New York; 40.7258283949511; -73.9958416166388; our courier; enabled
102842; Rubirosa bar; 235 Mulberry St; New York; 40.7258283949511; -73.9958416166388; our courier; disabled
102837; Rubirosa bar; 235 Mulberry St; New York; 40.7258283949; -73.9958416166; our courier; disabled
100923; Rubirosa bar; 235 Mulberry St; New York; 40.7258283949; -73.9958416166; our courier; disabled


Finally: we need to create the composite primary key of restaurants. In case if we want (sure, we do) to evaluate some statistics regarding restaurants we would not be able to do it due to unexisting of fields with the same values to the same restaurants. 

That's quite common issue for different CRM-systems: for example, a customer fills the landing web-page where specifies own name, surname and patronymic. After some time the customer provides this data for another page and doesn't leave patronymic field empty. In this case the customer could be counted twice as two separate users.

# New keys creation 

In [403]:
#we need to drop from original names the most frequent types of restaurant like 'bar', 'cafe', etc. as well as all special symbols 

black_list = [
    "pub", "bar", "bistro", "restaurant", "pizzeria", "club", "gastropub", "restopub", "chaihana", "chaihona", "café", "cafe", "caffe", "caffé"
]
deleted_list = [
    " ", "!", "№", ";", "%", ":", "?", "*", "(", ")", "[", "]", "{", "}", 
     "/", ".", "=", "+", "-", "_", "@", "#", "$", "^", "&", "\\", ",", "<", ">", "'"
]
          
#here we're going to clean up the original names
def string_cleaned(string: str, substitution_value: str) -> str: 
    string_splitted = (string.lower()).split(" ")

    string_cleaned = ""
    for idx, value in enumerate(string_splitted):
        if value in black_list:
            continue
        else:
            string_cleaned += value
            
    string_removed = ""
    for idx, value in enumerate(string_cleaned):
        if value in deleted_list:
            continue
        else:
            string_removed += value
            
    string_result = string_removed if string_removed is not None and string_removed != "" else substitution_value
    return string_result

Let's take a look at the result of this udf.

In [405]:
example_names = ["#GAMARJOBA restaurant", " Gastropub Rogor!!!   ", "Xar, bar", ""]

for idx, value in enumerate(example_names):
    print (value, string_cleaned(value, "unknown"), sep = " => ")

#GAMARJOBA restaurant => gamarjoba
 Gastropub Rogor!!!    => rogor
Xar, bar => xar
 => unknown


In [406]:
#due to possibility of different length of latitude and longitude we're going to assign each point to the cossresponding hexagon. 
#the attribute resolution below means the square of hexagon, you can read about it here: https://h3geo.org/docs/core-library/restable

def h3_hexagons(lat: float, lon: float, resolution: int, substitution_value: str) -> str:
    h3geo = h3.geo_to_h3(lat, lon, resolution) if lat is not None and lat != "" and lon is not None and lon != "" else substitution_value
    return h3geo

In [407]:
#now we want to create the composite key from a part of cleaned name and its hexagon. I'll explain additionally the reasons why I prefer to use a part of name instead of Levenshtein distance or some related libraries like FuzzyWuzzy, for example. 
def hash_result (string_cleaned:str, h3:str, start_point:int, end_point:int)->int:
    
    string_for_hash = string_cleaned[start_point:end_point]
    string_for_hash += h3
    hash_result = xx.xxh32()
    hash_result.update(string_for_hash)
    return hash_result.intdigest()

Finally, we create new identifier for each row and guess it would be same for different ids of the same physically existing restaurant. 

In case of some problems we will always substitute the algorithm result by the original id of the row.

In [408]:
df = df_restaurants.copy()
df["cleaned_name"] = ""
df["h3geo"] = ""
df["new_id"] = ""

for i in range(df.shape[0]):
    df.at[i, "cleaned_name"] = string_cleaned(df.at[i, "name"], df.at[i, "id"].astype("str"))
    df.at[i, "h3geo"] = h3_hexagons(df.at[i, "lat"], df.at[i, "lon"], 10, df.at[i, "id"].astype("str"))
    df.at[i, "new_id"] = hash_result(df.at[i, "cleaned_name"], df.at[i, "h3geo"], 1, 4)
    

Well, now we have the unique id for each physically existing restaurant regardless its data duplication or some string patterns issues.

In [410]:
df.sort_values(by = "new_id")[:5]

,id,name,address,lat,lon,city,delivery_type,enabled_status,cleaned_name,h3geo,new_id
19,100016,Upland,345 Park Ave S,40.744221,-73.985054,New York,taxi courier,enabled,upland,8a2a100d2107fff,216754633
5,101938,Upland,345 Park Ave S,40.744221,-73.985054,New York,restaurant courier,enabled,upland,8a2a100d2107fff,216754633
6,116843,Upland Café,345 Park Ave S,40.744221,-73.985054,New York,our courier,enabled,upland,8a2a100d2107fff,216754633
13,118329,Charlie Palmer Steak,135 42nd St,40.756730,-73.985130,NY,taxi courier,enabled,charliepalmersteak,8a2a100d679ffff,1110467645
12,100924,Charlie Palmer Steak NYC,135 W 42nd St,40.756727,-73.985135,New York,our courier,enabled,charliepalmersteaknyc,8a2a100d679ffff,1110467645


# Usage and Impact on Data

Now we can map all of different values of the same entitity to its unique key. 

In [411]:
df_new_rests = df.copy()
df_new_rests = df_new_rests.groupby("new_id").agg(
    id_all = ("id", "unique"), 
    name = ("name", "min"),
    name_all = ("name", "unique"),
    delivery_type_all = ("delivery_type", "unique"),
    enabled_status_all = ("enabled_status", "unique"),
    address = ("address", "min")
).reset_index()
df_new_rests

,new_id,id_all,name,name_all,delivery_type_all,enabled_status_all,address
0,216754633,"[101938, 116843, 100016]",Upland,"[Upland, Upland Café]","[restaurant courier, our courier, taxi courier]",[enabled],345 Park Ave S
1,1110467645,"[100924, 118329]",Charlie Palmer Steak,"[Charlie Palmer Steak NYC, Charlie Palmer Steak]","[our courier, taxi courier]",[enabled],135 42nd St
2,1246616891,[106832],The Smith,[The Smith],[our courier],[enabled],1900 Broadway
3,1773116187,"[121805, 102931]",The Smith,[The Smith],"[restaurant courier, our courier]",[enabled],956 2nd Ave
4,3640891658,"[101504, 114640, 114872, 101502]",The Smith,"[The Smith, The Smith's]","[restaurant courier, our courier, taxi courier]","[enabled, disabled]",55 3rd Ave
5,3935892236,"[122001, 102845, 102842, 102837, 100923]",Rubirosa,"[Rubirosa, Rubirosa Bar, Rubirosa bar]","[restaurant courier, our courier]","[enabled, disabled]",235 Mulberry St
6,3997671354,"[105964, 108235, 111924]",Bubby's,"[Bubby's, Bubbys]","[restaurant courier, our courier, taxi courier]",[enabled],120 Hudson St


That also has high impact on our business metrics. 

## 1) How many restaurants are enabled now regardless its delivery type? 

In [412]:
#basically we thought that all ids are unique
print("Incorrect answer", df_restaurants.query("enabled_status == 'enabled'").shape[0], sep = ": ")

#now it's clear that some of the are duplicated: the restaurant is active in case if at least one delivery type is available
statuses = np.array(df_new_rests["enabled_status_all"])
cnt = 0
for j, value in enumerate(statuses):
    if "enabled" in value:
        cnt += 1 
    else: 
        continue
        
print("Correct answer", cnt, sep = ": ")

Incorrect answer: 16
Correct answer: 7


## 2) What is the average value of orders count and GMV per restaurant (using all available date period from table with orders)? 

In [413]:
#let's calculate it using standart approach
df_join = pd.merge(df_restaurants, df_orders, how = "inner", left_on = "id", right_on = "restaurant_id")
df_join = df_join.rename(columns = {"id_y": "order_id", "id_x": "id"})

df_result = df_join.groupby(["restaurant_id", "name", "address"]).agg(
orders_count = ("order_id", "count"), 
gmv = ("order_amount", "sum")
).reset_index()

print ("Incorrect answers")
print ("Average orders count per restaurant", round((sum(df_result["orders_count"]) / df_result.shape[0]), 2), sep = ": ")
print ("Average GMV per restaurant", round((sum(df_result["gmv"]) / df_result.shape[0]), 2), sep = ": ")

print("")

#now the same but using unique keys
df_join_new = pd.merge(df, df_orders, how = "inner", left_on = "id", right_on = "restaurant_id")
df_join_new = df_join_new.rename(columns = {"id_y": "order_id", "id_x": "id"})
df_result_new = df_join_new.groupby(["new_id"]).agg(
name = ("name", "min")
, address = ("address", "min") 
, orders_count = ("order_id", "count")
, gmv = ("order_amount", "sum")
).reset_index()

print ("Correct answers")
print ("Average orders count per restaurant", round((sum(df_result_new["orders_count"]) / df_result_new.shape[0]), 2), sep = ": ")
print ("Average GMV per restaurant", round((sum(df_result_new["gmv"]) / df_result_new.shape[0]), 2), sep = ": ")


Incorrect answers
Average orders count per restaurant: 5.56
Average GMV per restaurant: 282.76

Correct answers
Average orders count per restaurant: 14.29
Average GMV per restaurant: 727.09


## 3) I need full statistics of restaurants performance.

Here we will have the result:

In [414]:
df_result_new.sort_values(by = "orders_count", ascending = False)

,new_id,name,address,orders_count,gmv
4,3640891658,The Smith,55 3rd Ave,25,1382.18
3,1773116187,The Smith,956 2nd Ave,22,1088.43
5,3935892236,Rubirosa,235 Mulberry St,17,1021.47
0,216754633,Upland,345 Park Ave S,16,545.72
6,3997671354,Bubby's,120 Hudson St,11,585.86
1,1110467645,Charlie Palmer Steak,135 42nd St,6,248.67
2,1246616891,The Smith,1900 Broadway,3,217.27


Instead of this one:

In [415]:
df_result.sort_values(by = "orders_count", ascending = False)

,restaurant_id,name,address,orders_count,gmv
16,121805,The Smith,956 2nd Ave,20,977.97
3,101938,Upland,345 Park Ave S,9,333.44
6,102845,Rubirosa Bar,235 Mulberry St,9,502.69
2,101504,The Smith,55 3rd Ave,8,336.86
12,114640,The Smith,55 Third Ave,7,413.39
14,116843,Upland Café,345 Park Ave S,7,212.28
1,101502,The Smith,55 3rd Ave,7,470.81
17,122001,Rubirosa,235 Mulberry St,4,288.65
15,118329,Charlie Palmer Steak,135 42nd St,4,173.82
8,105964,Bubby's,120 Hudson St,4,243.01
